# Notes
## Big Mistake in Prompt:
**Added the """ after a linebreak e.g.** <br>
Anwser: <br>
"""<br>
**Correct way:<br>**
Anwser:"""

## Max_output tokens to restrict the possible outputs to the max number of answers possibilities
## Temperature = 0


## CCNA 201-301 - 5 Shot like HELM with Answer format: Answer: ABC or Answer: A
Output of Phi-Model always empty, similiar config as mmlu (temp=0, max_output_toke = 2)
LLama 2 results are much worse in comp. to single shot with Correct Answer: ['A', 'B'] and temp =0.7 and no limit to output tokens

In [1]:
from templates import *

from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
import pandas as pd
from langchain import PromptTemplate
import re
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import random
import warnings
import numpy as np
import json
warnings.filterwarnings('ignore')
#Set the output limit to inf
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [2]:
MODEL_PATH = { "Mixtral-8x-7b": "../models/mixtral-8x7b-instruct-v0.1.Q4_K_M.gguf",
               "Phi-2": "../models/phi-2.Q5_K_M.gguf",
               "Llama2-70b": "../models/llama2_70b_chat_uncensored.Q5_K_M.gguf",
               "Yi-34b": "../models/yi-34b-200k.Q5_K_M.gguf",
               "Dolphin-2.5": "../models/dolphin-2.5-mixtral-8x7b.Q5_K_M.gguf",
              }

########### Set the model parameters here ############

#Parameter for changing the temperature of the model
TEMPERATURE = 0

#Parameter for max output tokens (for MMLU choose 1, since only Single choice)

MAX_OUTPUT_TOKENS = 2

########### Set the model parameters here ############


#Sampling rate determines how often a question is asked again if the answer format is wrong
MAX_SAMPLING_RATE = 5

#Set to 1 if you dont want to shuffle
NUM_OF_SHUFFLES = 5


########### Set the names for result / evaluation files here ############

#Set run name
RUN_NAME = "5_Shot_201_301_CCNA"

#Set output file name
OUTPUT_EVALUATION = "../results/5_Shot_201_301_CCNA/llm_5_Shot_201_301.pkl"

#Filename output evaluation detailed
OUTPUT_EVALUATION_DETAILED = "../results/5_Shot_201_301_CCNA/llm_prob_result_detailed_201_301_CCNA_5_Shot.pkl"

#Set filename of json file
OUTPUT_EVALUATION_JSON = "../results/5_Shot_201_301_CCNA/llm_prob_result_201_301_CCNA_5_Shot.json"

########### Set the names for result files here ############


########### Set the questionsbank here ############
#Set the questionsbank
QUESTIONS_BANK = "../data/201-301-CCNA.parquet"
########### Set the questionsbank here ############

########### Set the prompt template here ############
PROMPT_TEMPLATE = CCNA_5_SHOT_TEMPLATE
########### Set the prompt template here ############

In [3]:
#Save the parameters as a JSON file
parameters = {
    "RUN_NAME": RUN_NAME,  # Ersetzen Sie durch den tatsächlichen Wert
    "QUESTION_BANK": QUESTIONS_BANK,  # Ersetzen Sie durch den tatsächlichen Wert
    "MAX_SAMPLING_RATE": MAX_SAMPLING_RATE,  # Ersetzen Sie durch den tatsächlichen Wert
    "NUM_OF_SHUFFLES": NUM_OF_SHUFFLES,  # Ersetzen Sie durch den tatsächlichen Wert
    "FEW_SHOT_TEMPLATE": PROMPT_TEMPLATE,  # Ersetzen Sie durch den tatsächlichen Wert
    "TEMPERATURE": TEMPERATURE,  # Ersetzen Sie durch den tatsächlichen Wert
    "MAX_TOKENS": MAX_OUTPUT_TOKENS  # Ersetzen Sie durch den tatsächlichen Wert
}

# Speichern Sie das Wörterbuch als JSON-Datei
with open(OUTPUT_EVALUATION_JSON, 'w') as f:
    json.dump(parameters, f)

In [4]:
def extract_answer(answer):
    """Extracts the correct answers from the provided answer string.

    Args:
        answer: The answer string to extract the correct answers from.

    Returns:
        A list of correct answers (e.g., ['A', 'B']) if found, otherwise None. 
    """
    answer = re.sub(r'[\s\n.,]', '', answer)
    pattern = re.compile(r'^[A-Z,]*$')
    if re.match(pattern, answer):
        if ',' in answer:
            return None
        else:
            return list(answer)
    else:
        return None
    
def compare_answers(answerLLM, answer_exam):
    """Compares the extracted correct answers with the answers in answer_exam.

    Keyword arguments:
    answerLLM -- the list of answers extracted from the LLM answer
    answer_exam -- list of answers from the exam
    """
    # Convert answer_exam_list from letters to numbers
    answerLLM = [ord(answer) - 65 for answer in answerLLM]

    # Get number of correct answers in the exam
    num_of_correct_exam_answers = len(answer_exam)

    # Convert both lists to sets for efficient comparison
    answer_LLM_set = set(answerLLM)
    answer_exam_set = set(answer_exam)

    # Calculate the count of matching answers
    number_of_correct_llm_answers = len(answer_LLM_set.intersection(answer_exam_set))

    # Check if the number of answers given by the LLM is greater than the number of correct answers
    too_many_answ_given = False
    if len(answer_LLM_set) > num_of_correct_exam_answers:
        too_many_answ_given = True

    # Return a dictionary with the matching count and the number of correct answers
    return number_of_correct_llm_answers, too_many_answ_given

def format_choices_for_llm(choices):
    #Define the letters for the choices
    letters = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
    
    # Erstellen Sie den formatierten String
    formatted_choices = '\n'.join(f'{letters[i]}. {choice}' for i, choice in enumerate(choices))
    
    return formatted_choices

def evaluation_sampling(llm_answer, exam_Answers, num_of_correct_answer):
    """Analyse the answer given by the LLM and compare it with the exam answers.

    Keyword arguments:
    llm_answer -- the answer string given by the LLM
    exam_Answers -- the list of answers from the exam
    """

    answerLLM = extract_answer(llm_answer)
    if answerLLM is not None:
        num_of_correct_llm_Answers, too_many_answ = compare_answers(answerLLM, exam_Answers)
        if num_of_correct_llm_Answers == num_of_correct_answer and too_many_answ == False:
            answered_correctly = True
        else:
            answered_correctly = False 
        return num_of_correct_llm_Answers, answerLLM, too_many_answ, answered_correctly
    else:
         return -1


def evaluation(llm_output_dataframe):

    # Compute the number of total questions for each model
    number_of_questions = llm_output_dataframe.groupby('Model')['QuestionIndex'].count()
    
    #Number of fully correct answers given by the LLM
    correctly_answered = llm_output_dataframe.groupby('Model')['Answered_Correctly'].sum()

    #Number of incorrect answers given by the LLM
    incorrectly_answered = number_of_questions - correctly_answered

    #Amount of correct answers in the exam
    amount_correcct_exam_answers = llm_output_dataframe.groupby('Model')['NumberOfCorrectExamAnswers'].sum()

    #Amount of correct answers given by the LLM even if not fully correct
    amount_correcct_llm_answers = llm_output_dataframe.groupby('Model')['NumberOfCorrectLLMAnswers'].sum()
    
    #Calculation of Accuracy and Recall and f1 score
    accuracy = correctly_answered / number_of_questions
    accuracy_partial = amount_correcct_llm_answers / amount_correcct_exam_answers


    results_df = pd.DataFrame({
        'Number of Questions': number_of_questions,
        'Correctly Answered': correctly_answered,
        'Incorrectly Answered': incorrectly_answered,
        'Accuracy': accuracy,
        'Accuracy Partial': accuracy_partial,
    })

    results_df = results_df.reset_index()

    return results_df


def plot_evaluation(evaluation_df):
    """
    Plots evaluation metrics from a DataFrame containing columns:
        - 'Model'
        - 'Accuracy Mean', 'Accuracy Min', 'Accuracy Max'
        - 'Accuracy Partial Mean', 'Accuracy Partial Min', 'Accuracy Partial Max'
    """

    # Define a list of colors for the models
    colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']

    # Define bar width
    bar_width = 0.5  # Increase bar width for thicker bars

    # --- Subplot 1: Accuracy ---
    fig, axs = plt.subplots(1, 2, figsize=(14, 6))

    for i, model in enumerate(evaluation_df['Model']):
        bars = axs[0].bar(i + bar_width * i, evaluation_df.loc[i, 'Accuracy Mean'], bar_width, 
                   yerr=[[abs(evaluation_df.loc[i, 'Accuracy Mean'] - evaluation_df.loc[i, 'Accuracy Min'])], [abs(evaluation_df.loc[i, 'Accuracy Max'] - evaluation_df.loc[i, 'Accuracy Mean'])]],
                   label=model, color=colors[i % len(colors)], capsize=5)

    axs[0].set_ylabel('Accuracy (%)')
    axs[0].set_title('Accuracy Mean with Error Bars (Max and Min)', fontsize=12)
    axs[0].set_xticks([i + bar_width * i for i in range(len(evaluation_df['Model']))])
    axs[0].set_xticklabels(evaluation_df['Model'], rotation=45, ha='right', fontsize=10)
    axs[0].legend()
    axs[0].set_ylim([0, 1])
    axs[0].yaxis.set_major_locator(mtick.MultipleLocator(0.1))
    axs[0].yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
    axs[0].grid(True, linestyle='dotted', axis='y')

    # --- Subplot 2: Partial Accuracy ---
    for i, model in enumerate(evaluation_df['Model']):
        bars = axs[1].bar(i + bar_width * i, evaluation_df.loc[i, 'Accuracy Partial Mean'], bar_width,
                   yerr=[[abs(evaluation_df.loc[i, 'Accuracy Partial Mean'] - evaluation_df.loc[i, 'Accuracy Partial Min'])], [abs(evaluation_df.loc[i, 'Accuracy Partial Max'] - evaluation_df.loc[i, 'Accuracy Partial Mean'])]],
                   label=model, color=colors[i % len(colors)], capsize=5)

    axs[1].set_ylabel('Accuracy Partial (%)')
    axs[1].set_title('Accuracy Partial Mean with Error Bars (Max and Min)', fontsize=12)
    axs[1].set_xticks([i + bar_width * i for i in range(len(evaluation_df['Model']))])
    axs[1].set_xticklabels(evaluation_df['Model'], rotation=45, ha='right', fontsize=10)
    axs[1].legend()
    axs[1].set_ylim([0, 1])
    axs[1].yaxis.set_major_locator(mtick.MultipleLocator(0.1))
    axs[1].yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
    axs[1].grid(True, linestyle='dotted', axis='y')

    fig.tight_layout(pad=1.2)  # Decrease padding for closer plots
    plt.show()

def calculate_model_statistics(df):
    """
    Calculates statistics for each model in the DataFrame.
    
    Args:
    df (DataFrame): Input DataFrame containing evaluation metrics for different models.
    
    Returns:
    DataFrame: New DataFrame containing calculated statistics for each model.
    """
    model_stats = []
    for model, group_df in df.groupby('Model'):
        model_stat = {
            'Model': model,
            'Accuracy Mean': group_df['Accuracy'].mean(),
            'Accuracy Max': group_df['Accuracy'].max(),
            'Accuracy Min': group_df['Accuracy'].min(),
            'Accuracy STD': group_df['Accuracy'].std(),
            'Accuracy Partial Mean': group_df['Accuracy Partial'].mean(),
            'Accuracy Partial Max': group_df['Accuracy Partial'].max(),
            'Accuracy Partial Min': group_df['Accuracy Partial'].min(),
            'Accuracy Partial STD': group_df['Accuracy Partial'].std()
        }
        model_stats.append(model_stat)
    
    return pd.DataFrame(model_stats)


def shuffle_choices_and_update_answer(choices, answer):
    # Erstellen Sie eine Liste von Indizes und mischen Sie sie
    indices = list(range(len(choices)))
    random.shuffle(indices)
    
    # Verwenden Sie die gemischten Indizes, um die Auswahlmöglichkeiten und die Antwort zu aktualisieren
    shuffled_choices = [choices[i] for i in indices]
    updated_answer = [indices.index(a) for a in answer]  # Entfernen Sie +1, um 0-basierte Indizes zu verwenden
    
    return shuffled_choices, updated_answer

In [5]:
valid_question_answer = False  
#Create a dataframe with the size of NUM_OF_SHUFFLES which contains the dataframe llm_exam_result
shuffled_evalutation_df = pd.DataFrame(columns=[ 'Number of Questions','Correctly Answered','Incorrectly Answered','Accuracy','Accuracy Partial'])
questions  = pd.read_parquet(QUESTIONS_BANK)

#Take the first 20 questions
#questions = questions.head(20)

#questions = extract_answer_from_text_file("../data/questionbank_cisco_CCNP.txt")
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
prompt_template = PromptTemplate.from_template(PROMPT_TEMPLATE)

#Iterate over each model definied in the MODEL_PATH dictionary
for model, model_path in MODEL_PATH.items():
     #Load the model wiht LLamaCpp
    llm = LlamaCpp(
        model_path= model_path,
        n_gpu_layers=128,
        n_batch=1024,
        n_ctx=1100,
        temperature=TEMPERATURE,
        #top_p=0.9,
        max_tokens = MAX_OUTPUT_TOKENS,
        #callback_manager=callback_manager,
        verbose=False,  # Verbose is required to pass to the callback manager
    )

    chain = prompt_template | llm
    for shuffled_iteration in range(NUM_OF_SHUFFLES):
        llm_exam_result = pd.DataFrame(columns = ["Model", "QuestionIndex", "SamplingIndex", "NumberOfCorrectLLMAnswers", "NumberOfCorrectExamAnswers", "Ratio", "LLM_Answer", "Exam_Answers", "Answered_Correctly",  "Too_Many_answers"]) 
        #Iterate over each question in the question dataframe
        for index_question, row in questions.iterrows():
            question = row['question']
            choices = row['choices']
            answers = row['answer']
            num_of_correct_answer = len(answers)

            choices = format_choices_for_llm(choices)

            #Only if shuffle is enabled, shuffle the choices
            if shuffled_iteration > 0:
                choices, answers = shuffle_choices_and_update_answer(row['choices'], row['answer'])
                num_of_correct_answer = len(answers)
                choices = format_choices_for_llm(choices)
            #Empty the char_probabilities dictionary for each question
            char_probabilities = {}

            #Iterate over the maximum sampling rate
            for index_sampling in range(MAX_SAMPLING_RATE):
                # Invoke the chain with the question and choices              
                

                llm_answer = chain.invoke({"Exam_Question" : row['question'], "Exam_Choices" : choices})            
                # Check if the answer is in the expected format
                if extract_answer(llm_answer) is not None:
                    # Extract the correct answers from the LLM answer and analyse the answer
                    num_of_correct_llm_answer, answerLLm, too_many_answers, answered_correctly = evaluation_sampling(llm_answer, answers, num_of_correct_answer)
                    #Save the current sampling index -- How of the question has been asked until the answer was in the correct format
                    sample_Index = index_sampling
                    valid_question_answer = True
                    break
            
            #Depending on the result of the answer, add the result to the dataframe
            if not valid_question_answer:
                new_row = pd.DataFrame({"Model": [model], "QuestionIndex": [index_question], "SamplingIndex": [-1], "NumberOfCorrectLLMAnswers": [0], "NumberOfCorrectExamAnswers": [num_of_correct_answer], "Ratio": [-1], "LLM_Answer": [llm_answer], "Exam_Answers": [answers]})
                llm_exam_result = pd.concat([llm_exam_result, new_row], ignore_index=True)
            else:
                new_row = pd.DataFrame({"Model": [model], "QuestionIndex": [index_question], "SamplingIndex": [sample_Index], "NumberOfCorrectLLMAnswers": [num_of_correct_llm_answer], "NumberOfCorrectExamAnswers": [num_of_correct_answer], "Ratio": [num_of_correct_llm_answer/num_of_correct_answer], "LLM_Answer": [answerLLm], "Exam_Answers": [answers], "Answered_Correctly" : [answered_correctly], "Too_Many_answers": [too_many_answers]})
                llm_exam_result = pd.concat([llm_exam_result, new_row], ignore_index=True)
                valid_question_answer = False
        answered_correctly = False
        #Concat the the dataframe returned by evaulation to one dataframe
        display(llm_exam_result)
        llm_exam_result.to_pickle(f"../data/{model}_shuffled_{shuffled_iteration}_mmlu.pkl")
        evaluation_df = evaluation(llm_exam_result)
        #Concat the evaluation dataframe to the complete dataframe
        shuffled_evalutation_df = pd.concat([shuffled_evalutation_df, evaluation_df], ignore_index=True)
        display(shuffled_evalutation_df)

#plot_evaluation(shuffled_evalutation_df)
model_statistics = calculate_model_statistics(shuffled_evalutation_df)
display(model_statistics)
plot_evaluation(model_statistics)
shuffled_evalutation_df.to_pickle(OUTPUT_EVALUATION_DETAILED)
model_statistics.to_pickle(OUTPUT_EVALUATION)


ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 2 CUDA devices:
  Device 0: NVIDIA A100-SXM4-40GB, compute capability 8.0, VMM: yes
  Device 1: NVIDIA A100-SXM4-40GB, compute capability 8.0, VMM: yes
llama_model_loader: loaded meta data with 26 key-value pairs and 995 tensors from ../models/mixtral-8x7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mixtral-8x7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          ll

,Model,QuestionIndex,SamplingIndex,NumberOfCorrectLLMAnswers,NumberOfCorrectExamAnswers,Ratio,LLM_Answer,Exam_Answers,Answered_Correctly,Too_Many_answers
0,Mixtral-8x-7b,0,0,2,2,1.000000,"[A, D]","[0, 3]",True,False
1,Mixtral-8x-7b,1,0,1,1,1.000000,[A],[0],True,False
2,Mixtral-8x-7b,2,0,1,1,1.000000,[A],[0],True,False
3,Mixtral-8x-7b,3,0,0,1,0.000000,[A],[2],False,False
4,Mixtral-8x-7b,4,0,1,1,1.000000,[C],[2],True,False
5,Mixtral-8x-7b,5,0,0,1,0.000000,[B],[2],False,False
6,Mixtral-8x-7b,6,0,2,2,1.000000,"[D, E]","[3, 4]",True,False
7,Mixtral-8x-7b,7,0,0,1,0.000000,[A],[2],False,False
8,Mixtral-8x-7b,8,0,0,1,0.000000,[C],[3],False,False
9,Mixtral-8x-7b,9,0,0,1,0.000000,[D],[1],False,False


,Number of Questions,Correctly Answered,Incorrectly Answered,Accuracy,Accuracy Partial,Model
0,204,124,80,0.607843,0.662651,Mixtral-8x-7b


,Model,QuestionIndex,SamplingIndex,NumberOfCorrectLLMAnswers,NumberOfCorrectExamAnswers,Ratio,LLM_Answer,Exam_Answers,Answered_Correctly,Too_Many_answers
0,Mixtral-8x-7b,0,0,1,2,0.500000,"[A, C]","[3, 2]",False,False
1,Mixtral-8x-7b,1,0,1,1,1.000000,[B],[1],True,False
2,Mixtral-8x-7b,2,0,0,1,0.000000,[D],[1],False,False
3,Mixtral-8x-7b,3,0,0,1,0.000000,[A],[2],False,False
4,Mixtral-8x-7b,4,0,1,1,1.000000,[A],[0],True,False
5,Mixtral-8x-7b,5,0,0,1,0.000000,[D],[2],False,False
6,Mixtral-8x-7b,6,0,2,2,1.000000,"[A, B]","[1, 0]",True,False
7,Mixtral-8x-7b,7,0,1,1,1.000000,[B],[1],True,False
8,Mixtral-8x-7b,8,0,0,1,0.000000,[D],[2],False,False
9,Mixtral-8x-7b,9,0,0,1,0.000000,[D],[2],False,False


,Number of Questions,Correctly Answered,Incorrectly Answered,Accuracy,Accuracy Partial,Model
0,204,124,80,0.607843,0.662651,Mixtral-8x-7b
1,204,116,88,0.568627,0.634538,Mixtral-8x-7b


...

llama_model_loader: loaded meta data with 22 key-value pairs and 543 tensors from ../models/yi-34b-200k.Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = 01-ai_yi-34b-200k
llama_model_loader: - kv   2:                       llama.context_length u32              = 200000
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 7168
llama_model_loader: - kv   4:                          llama.block_count u32              = 60
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 20480
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head

,Model,QuestionIndex,SamplingIndex,NumberOfCorrectLLMAnswers,NumberOfCorrectExamAnswers,Ratio,LLM_Answer,Exam_Answers,Answered_Correctly,Too_Many_answers
0,Yi-34b,0,0,0,2,0.000000,[],"[0, 3]",False,False
1,Yi-34b,1,0,0,1,0.000000,[],[0],False,False
2,Yi-34b,2,0,0,1,0.000000,[],[0],False,False
3,Yi-34b,3,0,0,1,0.000000,[],[2],False,False
4,Yi-34b,4,0,0,1,0.000000,[B],[2],False,False
5,Yi-34b,5,0,0,1,0.000000,[],[2],False,False
6,Yi-34b,6,0,0,2,0.000000,[],"[3, 4]",False,False
7,Yi-34b,7,0,1,1,1.000000,[C],[2],True,False
8,Yi-34b,8,0,0,1,0.000000,[B],[3],False,False
9,Yi-34b,9,-1,0,1,-1.000000,17,[1],NaN,NaN


,Number of Questions,Correctly Answered,Incorrectly Answered,Accuracy,Accuracy Partial,Model
0,204,124,80,0.607843,0.662651,Mixtral-8x-7b
1,204,116,88,0.568627,0.634538,Mixtral-8x-7b
2,204,119,85,0.583333,0.626506,Mixtral-8x-7b
3,204,111,93,0.544118,0.60241,Mixtral-8x-7b
4,204,123,81,0.602941,0.650602,Mixtral-8x-7b
5,204,0,204,0.0,0.0,Phi-2
6,204,0,204,0.0,0.0,Phi-2
7,204,0,204,0.0,0.0,Phi-2
8,204,0,204,0.0,0.0,Phi-2
9,204,0,204,0.0,0.0,Phi-2


,Model,QuestionIndex,SamplingIndex,NumberOfCorrectLLMAnswers,NumberOfCorrectExamAnswers,Ratio,LLM_Answer,Exam_Answers,Answered_Correctly,Too_Many_answers
0,Yi-34b,0,0,0,2,0.0,[],"[0, 1]",False,False
1,Yi-34b,1,0,1,1,1.0,[B],[1],True,False
2,Yi-34b,2,0,0,1,0.0,[],[3],False,False
3,Yi-34b,3,0,0,1,0.0,[],[1],False,False
4,Yi-34b,4,0,0,1,0.0,[B],[3],False,False
5,Yi-34b,5,0,1,1,1.0,[C],[2],True,False
6,Yi-34b,6,0,0,2,0.0,[],"[3, 4]",False,False
7,Yi-34b,7,0,0,1,0.0,[],[2],False,False
8,Yi-34b,8,0,0,1,0.0,[],[0],False,False
9,Yi-34b,9,-1,0,1,-1.0,19,[3],NaN,NaN


,Number of Questions,Correctly Answered,Incorrectly Answered,Accuracy,Accuracy Partial,Model
0,204,124,80,0.607843,0.662651,Mixtral-8x-7b
1,204,116,88,0.568627,0.634538,Mixtral-8x-7b
2,204,119,85,0.583333,0.626506,Mixtral-8x-7b
3,204,111,93,0.544118,0.60241,Mixtral-8x-7b
4,204,123,81,0.602941,0.650602,Mixtral-8x-7b
5,204,0,204,0.0,0.0,Phi-2
6,204,0,204,0.0,0.0,Phi-2
7,204,0,204,0.0,0.0,Phi-2
8,204,0,204,0.0,0.0,Phi-2
9,204,0,204,0.0,0.0,Phi-2


In [ ]:
for model, model_path in MODEL_PATH.items():
    print(f"Model: {model}")
    for shuffled_iteration in range(NUM_OF_SHUFFLES):
        llm_exam_result = pd.read_pickle(f"../data/{model}_shuffled_{shuffled_iteration}_201_301.pkl")
        evaluation_df = evaluation(llm_exam_result)
        #Concat the evaluation dataframe to the complete dataframe
        shuffled_evalutation_df = pd.concat([shuffled_evalutation_df, evaluation_df], ignore_index=True)
model_statistics = calculate_model_statistics(shuffled_evalutation_df)
display(model_statistics)
plot_evaluation(model_statistics)
shuffled_evalutation_df.to_pickle(OUTPUT_EVALUATION_DETAILED)
model_statistics.to_pickle(OUTPUT_EVALUATION)